# Stacked RNN + Softmax Layer

In [1]:
import tensorflow as tf
import numpy as np

In [6]:
sentence = (" if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

# Hyper parameters
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

0  if you wa -> if you wan
1 if you wan -> f you want
2 f you want ->  you want 
3  you want  -> you want t
4 you want t -> ou want to
5 ou want to -> u want to 
6 u want to  ->  want to b
7  want to b -> want to bu
8 want to bu -> ant to bui
9 ant to bui -> nt to buil
10 nt to buil -> t to build
11 t to build ->  to build 
12  to build  -> to build a
13 to build a -> o build a 
14 o build a  ->  build a s
15  build a s -> build a sh
16 build a sh -> uild a shi
17 uild a shi -> ild a ship
18 ild a ship -> ld a ship,
19 ld a ship, -> d a ship, 
20 d a ship,  ->  a ship, d
21  a ship, d -> a ship, do
22 a ship, do ->  ship, don
23  ship, don -> ship, don'
24 ship, don' -> hip, don't
25 hip, don't -> ip, don't 
26 ip, don't  -> p, don't d
27 p, don't d -> , don't dr
28 , don't dr ->  don't dru
29  don't dru -> don't drum
30 don't drum -> on't drum 
31 on't drum  -> n't drum u
32 n't drum u -> 't drum up
33 't drum up -> t drum up 
34 t drum up  ->  drum up p
35  drum up p -> drum up pe
36

In [7]:
with tf.variable_scope('long_char_multirnn', reuse=tf.AUTO_REUSE) as scope:
    X = tf.placeholder(tf.int32, [None, sequence_length])
    Y = tf.placeholder(tf.int32, [None, sequence_length])

    x_one_hot = tf.one_hot(X, num_classes)   # one_hot 사용할 때는 항상 shape 확인 필요

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([cell]*2, state_is_tuple=True)

    initial_state = cell.zero_state(batch_size, tf.float32)

    outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)
    
    # softmax layer
    x_for_softmax = tf.reshape(outputs, [-1, hidden_size])
    softmax_w = tf.get_variable('softmax_w', [hidden_size, num_classes])
    softmax_b = tf.get_variable('softmax_b', [num_classes])
    outputs = tf.matmul(x_for_softmax, softmax_w) + softmax_b
    #
    
    outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
    
    weights = tf.ones([batch_size, sequence_length])
    sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

    loss = tf.reduce_mean(sequence_loss)
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(500):
            _, l, results = sess.run([train, loss, outputs], feed_dict={X: dataX, Y: dataY})
            
            for j, result in enumerate(results):
                index = np.argmax(result, axis=1)

                print(i, j, ''.join([char_set[t] for t in index]), l)
            
        results = sess.run(outputs, feed_dict={X: dataX})

        for j, result in enumerate(results):
            index = np.argmax(result, axis=1)

            if j is 0:  # print all for the first result to make a sentence
                print(''.join([char_set[t] for t in index]), end='')
            else:
                print(char_set[index[-1]], end='')

0 0 rrllllllll 3.2480655
0 1 uullllllll 3.2480655
0 2 llllllllll 3.2480655
0 3 rrllllllur 3.2480655
0 4 uuuulluuuu 3.2480655
0 5 uuuluuuuuu 3.2480655
0 6 ullrruruuu 3.2480655
0 7 rrrrrrruuu 3.2480655
0 8 rrruuuuuuu 3.2480655
0 9 uuuuuuuuuu 3.2480655
0 10 uuuuuuuuul 3.2480655
0 11 uuuuuuuull 3.2480655
0 12 rruuuuulll 3.2480655
0 13 uuuuuullll 3.2480655
0 14 uuuuulllll 3.2480655
0 15 rrllllllrr 3.2480655
0 16 uuuuulurrr 3.2480655
0 17 uululurrrr 3.2480655
0 18 uuuurrrrrr 3.2480655
0 19 rrrrrrrrrr 3.2480655
0 20 urrrrrrrrr 3.2480655
0 21 rrrrrrrrrr 3.2480655
0 22 urrrruurru 3.2480655
0 23 rrrrrrrrru 3.2480655
0 24 uruuurruuu 3.2480655
0 25 ruuurruuuu 3.2480655
0 26 uuulruuuur 3.2480655
0 27 uurruurrrr 3.2480655
0 28 rrrrurrrrr 3.2480655
0 29 rruururrrr 3.2480655
0 30 uuuuurrrrr 3.2480655
0 31 uuuuurrrrr 3.2480655
0 32 urrrrrrrrr 3.2480655
0 33 rrrrrrrrrl 3.2480655
0 34 uuurrrrlll 3.2480655
0 35 rrrrrrllll 3.2480655
0 36 urrlrlllll 3.2480655
0 37 rlllllllll 3.2480655
0 38 ulllllllll 3.2480

2 108            3.1174855
2 109            3.1174855
2 110            3.1174855
2 111            3.1174855
2 112            3.1174855
2 113            3.1174855
2 114            3.1174855
2 115            3.1174855
2 116            3.1174855
2 117            3.1174855
2 118            3.1174855
2 119            3.1174855
2 120            3.1174855
2 121            3.1174855
2 122            3.1174855
2 123            3.1174855
2 124            3.1174855
2 125            3.1174855
2 126            3.1174855
2 127            3.1174855
2 128            3.1174855
2 129            3.1174855
2 130            3.1174855
2 131            3.1174855
2 132            3.1174855
2 133            3.1174855
2 134            3.1174855
2 135            3.1174855
2 136            3.1174855
2 137            3.1174855
2 138            3.1174855
2 139            3.1174855
2 140            3.1174855
2 141            3.1174855
2 142            3.1174855
2 143            3.1174855
2 144            3.1174855
2

4 141 eeeeeeeeee 2.8989182
4 142 eeeeeeeeee 2.8989182
4 143 eeeeeeeeee 2.8989182
4 144 eeeeeeeeee 2.8989182
4 145 eeeeeeeeee 2.8989182
4 146 eeeeeeeeee 2.8989182
4 147 eeeeeeeeee 2.8989182
4 148 eeeeeeeeee 2.8989182
4 149 eeeeeeeeee 2.8989182
4 150 eeeeeeeeee 2.8989182
4 151 eeeeeeeeee 2.8989182
4 152 eeeeeeeeee 2.8989182
4 153 eeeeeeeeee 2.8989182
4 154 eeeeeeeeee 2.8989182
4 155 eeeeeeeeee 2.8989182
4 156 eeeeeeeeee 2.8989182
4 157 eeeeeeeeee 2.8989182
4 158 eeeeeeeeee 2.8989182
4 159 eeeeeeeeee 2.8989182
4 160 eeeeeeeeee 2.8989182
4 161 eeeeeeeeee 2.8989182
4 162 eeeeeeeeee 2.8989182
4 163 eeeeeeeeee 2.8989182
4 164 eeeeeeeeee 2.8989182
4 165 eeeeeeeeee 2.8989182
4 166 eeeeeeeeee 2.8989182
4 167 eeeeeeeeee 2.8989182
4 168 eeeeeeeeee 2.8989182
4 169 eeeeeeeeee 2.8989182
4 170 eeeeeeeeee 2.8989182
5 0 o          2.8785036
5 1 o          2.8785036
5 2 o          2.8785036
5 3 o          2.8785036
5 4 o          2.8785036
5 5            2.8785036
5 6 o          2.8785036
5 7 o          

7 71            2.8580194
7 72            2.8580194
7 73            2.8580194
7 74            2.8580194
7 75            2.8580194
7 76            2.8580194
7 77            2.8580194
7 78            2.8580194
7 79            2.8580194
7 80            2.8580194
7 81            2.8580194
7 82            2.8580194
7 83            2.8580194
7 84            2.8580194
7 85            2.8580194
7 86            2.8580194
7 87            2.8580194
7 88            2.8580194
7 89            2.8580194
7 90            2.8580194
7 91            2.8580194
7 92            2.8580194
7 93            2.8580194
7 94            2.8580194
7 95            2.8580194
7 96            2.8580194
7 97            2.8580194
7 98            2.8580194
7 99            2.8580194
7 100            2.8580194
7 101            2.8580194
7 102            2.8580194
7 103            2.8580194
7 104            2.8580194
7 105            2.8580194
7 106            2.8580194
7 107            2.8580194
7 108            2.8580194
7 1

9 160            2.8463242
9 161            2.8463242
9 162            2.8463242
9 163            2.8463242
9 164            2.8463242
9 165            2.8463242
9 166            2.8463242
9 167            2.8463242
9 168            2.8463242
9 169            2.8463242
9 170            2.8463242
10 0            2.8395247
10 1            2.8395247
10 2            2.8395247
10 3            2.8395247
10 4            2.8395247
10 5            2.8395247
10 6            2.8395247
10 7            2.8395247
10 8            2.8395247
10 9            2.8395247
10 10            2.8395247
10 11            2.8395247
10 12            2.8395247
10 13            2.8395247
10 14            2.8395247
10 15            2.8395247
10 16            2.8395247
10 17            2.8395247
10 18            2.8395247
10 19            2.8395247
10 20            2.8395247
10 21            2.8395247
10 22            2.8395247
10 23            2.8395247
10 24            2.8395247
10 25            2.8395247
10 26      

...

488 127 nh them to 0.23075029
488 128 h them to  0.23075029
488 129 ethem to l 0.23075029
488 130 toem to lo 0.23075029
488 131 her ta lon 0.23075029
488 132 er to long 0.23075029
488 133 r ta long  0.23075029
488 134  ta long f 0.23075029
488 135 to bong fo 0.23075029
488 136 h cong for 0.23075029
488 137  long for  0.23075029
488 138 tong for t 0.23075029
488 139 eng for th 0.23075029
488 140  ' for the 0.23075029
488 141 d for the  0.23075029
488 142 nfor the e 0.23075029
488 143 tor the en 0.23075029
488 144 or the end 0.23075029
488 145   the endl 0.23075029
488 146  toe endle 0.23075029
488 147 toemendles 0.23075029
488 148 herendless 0.23075029
488 149 erendless  0.23075029
488 150 rtndless i 0.23075029
488 151 tndless im 0.23075029
488 152 rsless imm 0.23075029
488 153 d ess imme 0.23075029
488 154  ess immen 0.23075029
488 155 ecs immens 0.23075029
488 156 rs immensi 0.23075029
488 157 siimmensit 0.23075029
488 158 sammensity 0.23075029
488 159 tmmensity  0.23075029
488 160 gm

491 90   them tas 0.230575
491 91 dthem task 0.230575
491 92 toem tasks 0.230575
491 93 her tasks  0.230575
491 94 er tosks a 0.230575
491 95   tasks an 0.230575
491 96  tasks and 0.230575
491 97 tosks and  0.230575
491 98 hsks and w 0.230575
491 99 nss and wo 0.230575
491 100 ss and wor 0.230575
491 101 s and work 0.230575
491 102 sand work, 0.230575
491 103 tnd dork,  0.230575
491 104 nd dork, b 0.230575
491 105 d dork, bu 0.230575
491 106  aork, but 0.230575
491 107 tork, but  0.230575
491 108 ook, but r 0.230575
491 109  k, but ra 0.230575
491 110  , but rat 0.230575
491 111 s but rath 0.230575
491 112  dut rathe 0.230575
491 113 tui rather 0.230575
491 114 ui rather  0.230575
491 115 i rather t 0.230575
491 116 haather te 0.230575
491 117 tather tea 0.230575
491 118  ther teac 0.230575
491 119 nher teach 0.230575
491 120 her toach  0.230575
491 121 er toach t 0.230575
491 122   toach th 0.230575
491 123  thach the 0.230575
491 124 toach them 0.230575
491 125 hach them  0.230575
49

494 77 d don't as 0.23059867
494 78  aon't ass 0.23059867
494 79 ton't dssi 0.23059867
494 80  n't dssig 0.23059867
494 81 n't dssign 0.23059867
494 82 dt dssign  0.23059867
494 83 t dssign t 0.23059867
494 84  dssign th 0.23059867
494 85 tnsign the 0.23059867
494 86 nkign them 0.23059867
494 87 sign them  0.23059867
494 88 stn them t 0.23059867
494 89 pn them ta 0.23059867
494 90   them tas 0.23059867
494 91 dthem task 0.23059867
494 92 toem tosks 0.23059867
494 93  er tosks  0.23059867
494 94 er tosks a 0.23059867
494 95   tosks an 0.23059867
494 96  tosks and 0.23059867
494 97 tosks and  0.23059867
494 98  sks and w 0.23059867
494 99 nks and wo 0.23059867
494 100 ss and wor 0.23059867
494 101 s and work 0.23059867
494 102 sand work, 0.23059867
494 103 tnd dork,  0.23059867
494 104 nd dork, b 0.23059867
494 105 d dork, bu 0.23059867
494 106  aork, but 0.23059867
494 107 took, but  0.23059867
494 108 ook, but r 0.23059867
494 109 n , but ra 0.23059867
494 110  , but rat 0.23059867
494

497 64  lect wood 0.23046036
497 65 eect wood  0.23046036
497 66 e t wood a 0.23046036
497 67 mt wood an 0.23046036
497 68 t wood and 0.23046036
497 69  wood and  0.23046036
497 70 tood and d 0.23046036
497 71 ord and do 0.23046036
497 72  d and don 0.23046036
497 73   and don' 0.23046036
497 74  a d don't 0.23046036
497 75 tnd don't  0.23046036
497 76 nd don't a 0.23046036
497 77 d don't as 0.23046036
497 78  aon't ass 0.23046036
497 79 ton't dssi 0.23046036
497 80  n't dssig 0.23046036
497 81  't dssign 0.23046036
497 82 dt dssign  0.23046036
497 83 t dssign t 0.23046036
497 84  wssign th 0.23046036
497 85 tnsign the 0.23046036
497 86 nkign them 0.23046036
497 87 s gn them  0.23046036
497 88 stn them t 0.23046036
497 89 fn them ta 0.23046036
497 90 e them tas 0.23046036
497 91 dthem task 0.23046036
497 92 toem tosks 0.23046036
497 93  em tosks  0.23046036
497 94 er tosks a 0.23046036
497 95 m tosks an 0.23046036
497 96  tosks and 0.23046036
497 97 tosks and  0.23046036
497 98  sks an

tf you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.